In [1]:
import json
import string
import numpy as np
import math
import sys
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from functools import lru_cache

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\soham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\soham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [120]:
# utility
table = str.maketrans('', '', string.punctuation) # punctuation table
en_stop = set(stopwords.words('english')).union(set(string.punctuation)).union(["n't", "'s"]) # add punctuations and additional stopwords

ps = PorterStemmer()
p_stemmer = lru_cache(maxsize=None)(ps.stem)

def json_writer(data, fname):
    """
        Write multiple json files
        Args:
            data: list(dict): list of dictionaries to be written as json
            fname: str: output file name
    """
    with open(fname, mode="w") as fp:
        for line in data:
            json.dump(line, fp)
            fp.write("\n")


def json_reader(fname):
    """
        Read multiple json files
        Args:
            fname: str: input file
        Returns:
            generator: iterator over documents 
    """
    for line in open(fname, mode="r"):
        yield json.loads(line)


def _stem(doc, p_stemmer, en_stop, return_tokens):
    tokens = word_tokenize(doc.lower())
    stopped_tokens = filter(lambda token: token not in en_stop, tokens)
    stemmed_tokens = map(lambda token: p_stemmer(token), stopped_tokens)
    if not return_tokens:
        return ' '.join(stemmed_tokens)
    return list(stemmed_tokens)


def getStemmedDocuments(docs, return_tokens=True):
    """
        Args:
            docs: str/list(str): document or list of documents that need to be processed
            return_tokens: bool: return a re-joined string or tokens
        Returns:
            str/list(str): processed document or list of processed documents
        Example: 
            new_text = "It is important to by very pythonly while you are pythoning with python.
                All pythoners have pythoned poorly at least once."
            print(getStemmedDocuments(new_text))
        Reference: https://pythonprogramming.net/stemming-nltk-tutorial/
    """
    en_stop = set(stopwords.words('english')).union(set(string.punctuation)).union(["n't", "'s"]) # add punctuations and additional stopwords

    ps = PorterStemmer()
    p_stemmer = lru_cache(maxsize=None)(ps.stem)
    if isinstance(docs, list):
        output_docs = []
        doc_count = 0
        for item in docs:
            doc_count += 1
            if doc_count % 50000 == 0:
                print("doc", doc_count)
            output_docs.append(_stem(item, p_stemmer, en_stop, return_tokens))
        return output_docs
    else:
        return _stem(docs, p_stemmer, en_stop, return_tokens)

def indicator(exp):
    if exp:
        return 1
    else:
        return 0

In [125]:
new_text = "It is important to by very pythonly Soham APPle apleLL while you are pythoning with python. ll pythoner,  s have pythoned poorly at least once. don't isn't shizdnt's but home's"
ss = np.empty((1), dtype='S10')
ss[0] = new_text
# print(getStemmedDocuments(new_text))
print(ss)
# print(getStemmedDocuments(ss))
# word_tokenize(new_text)
# p = set(string.punctuation)
# print(p.union(["n't"]))
# print(set(stopwords.words('english')).union(set(string.punctuation)).union(["n't", "'s"]))
e,r = [],[]
e

[b'It is impo']


[]

In [130]:

train_json = "C:/IITD/sem5/col774-ml/col774_yelp_data/col774_yelp_data/train.json"
test_json = "C:/IITD/sem5/col774-ml/col774_yelp_data/col774_yelp_data/test.json"

vocabulary = {}
x = []
y = []

i = -1
for review in json_reader(train_json):
    i += 1
    if i % 50000 == 0:
        print("train", i, len(vocabulary))
    y.append(int(review["stars"]))
    
    xi = []
    words = _stem(review["text"], p_stemmer, en_stop, return_tokens=True)
    
    # bi-grams
    for w in range(len(words)-1):
        bg = (words[w], words[w+1])
        if bg not in vocabulary:
            vocabulary[bg] = len(vocabulary) + 1
        xi.append(vocabulary[bg])
    x.append(xi)

x_test = []
y_test = []

i = -1
for review in json_reader(test_json):
    i += 1
    if i % 50000 == 0:
        print("test", i, len(vocabulary))
    y_test.append(int(review["stars"]))
    
    xi = []
    words = _stem(review["text"], p_stemmer, en_stop, return_tokens=True)
    
    # bi-grams
    for w in range(len(words)-1):
        bg = (words[w], words[w+1])
        if bg not in vocabulary:
            vocabulary[bg] = len(vocabulary) + 1
        xi.append(vocabulary[bg])
    x_test.append(xi)

train 0 0
train 50000 1208884
train 100000 2004812
train 150000 2672199
train 200000 3264531
train 250000 3809075
train 300000 4319510
train 350000 4794161
train 400000 5245203
train 450000 5681575
train 500000 6100473
test 0 6379340
test 50000 6770362
test 100000 7147753


In [138]:
m = len(y)
m_test = len(y_test)

In [71]:
vocabulary = {}

# load training data
with open("C:/IITD/sem5/col774-ml/col774_yelp_data/col774_yelp_data/train.json") as train_file:
    m = sum(1 for line in train_file)
    y = np.zeros(m, np.int)
    x = []

with open("C:/IITD/sem5/col774-ml/col774_yelp_data/col774_yelp_data/train.json") as train_file:
    i = -1
    for line in train_file:
        i += 1
        if i % 10000 == 0:
            print("train", i, len(vocabulary))
        review = json.loads(line)
        y[i] = int(review["stars"])
        
        xi = []
        words = getStemmedDocuments(review["text"])
        
        # bi-grams
        for w in range(len(words)-1):
            bg = (words[w], words[w+1])
            if bg not in vocabulary:
                vocabulary[bg] = len(vocabulary) + 1
            xi.append(vocabulary[bg])
        x.append(xi)

# load test data
with open("C:/IITD/sem5/col774-ml/col774_yelp_data/col774_yelp_data/test.json") as test_file:
    m_test = sum(1 for line in test_file)
    y_test = np.zeros(m_test, np.int)
    x_test = []

with open("C:/IITD/sem5/col774-ml/col774_yelp_data/col774_yelp_data/test.json") as test_file:
    i = -1
    for line in test_file:
        i += 1
        if i % 50000 == 0:
            print("test", i, len(vocabulary))
        review = json.loads(line)
        y_test[i] = int(review["stars"])
        
        xi = []
        words = []
        for word in getStemmedDocuments(review["text"]):
            word = word.translate(table) # remove punctuations
            if word:
                words.append(word)

        # bi-grams
        for i in range(len(words)-1):
            bg = (words[i], words[i+1])
            if bg not in vocabulary:
                vocabulary[bg] = len(vocabulary) + 1
            xi.append(vocabulary[bg])
        x_test.append(xi)

train 0 0
train 10000 341225


KeyboardInterrupt: 

In [134]:
r = 5 # number of classes
V = len(vocabulary)

# y takes values in {1, 2, ..., r}; parameterized by phi
# x takes values in {1, 2, ..., V}; parameterized by theta

In [135]:
# evaluate phi
phi = np.zeros(r)
for i in range(m):
    k = y[i] - 1
    phi[k] += 1
phi = phi / m

# evaluate theta
theta_numerator = np.zeros((V, r))
theta_denominator = np.zeros((r))
for i in range(m):
    if i % 50000 == 0:
        print(i)
    ni = len(x[i])
    k = y[i] - 1
    for j in range(ni):
        l = x[i][j] - 1
        theta_numerator[l][k] += 1
    theta_denominator[k] += ni
    

theta = np.zeros((V, r))
for j in range(V):
    for k in range(r):
        theta[j][k] = (theta_numerator[j][k] + 1) / (theta_denominator[k] + V)

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000


In [144]:
np.sum(theta[:, 1])

0.9999999999999994

In [136]:
def predict(x, phi, theta):
    """
        Returns: Most expected class label
    """
    max_prob = -math.inf
    argmax_k = -1
    for k in range(r):
        n = len(x)
        summation = 0
        for i in range(n):
            l = x[i] - 1
            summation += np.log(theta[l][k])
        prob_k = summation + np.log(phi[k])
        if prob_k > max_prob:
            max_prob = prob_k
            argmax_k = k
    return argmax_k + 1

In [141]:
# train set accuracy

train_count = 0
for i in range(m):
    if i % 50000 == 0:
        print("train", i)
    train_count += indicator(predict(x[i], phi, theta) == y[i])

print("Train set accuracy", (train_count / m) * 100)

train 0
train 50000
train 100000
train 150000
train 200000
train 250000
train 300000
train 350000
train 400000
train 450000
train 500000
Train set accuracy 89.5231008540361


In [139]:
# test set accuracy

test_predictions = np.zeros(m_test, np.int)
test_count = 0
for i in range(m_test):
    if i % 50000 == 0:
        print("test", i)
    test_predictions[i] = predict(x_test[i], phi, theta)
    test_count += indicator(test_predictions[i] == y_test[i])

print("Test set accuracy", (test_count / m_test) * 100)

test 0
test 50000
test 100000
Test set accuracy 63.17997576990383


In [ ]:
# random prediction and majority prediction

majority_k = np.argmax(np.bincount(y)) # max occuring class

random_count = 0
majority_count = 0
for i in range(len(y_test)):
    random_count += indicator(y_test[i] == np.random.randint(r))
    majority_count += indicator(y_test[i] == majority_k)

print("Random prediction accuracy", (random_count / m_test) * 100)
print("Majority prediction accuracy", (majority_count / m_test) * 100)

In [140]:
# confusion matrix
confusion_matrix = np.zeros((r, r), np.int)

for i in range(m_test):
    confusion_matrix[test_predictions[i]-1][y_test[i]-1] += 1

print(confusion_matrix)

[[16604  4130  1659   749  1162]
 [  739   780   222    70    74]
 [  659  1516  1730   454   181]
 [ 1327  3478  8863 16145  8181]
 [  840   934  2057 11940 49224]]
